#### Connect to mysql server

In [0]:
driver = "com.mysql.cj.jdbc.Driver"

database_host = "movie-server.mysql.database.azure.com"
database_port = "3306" # update if you use a non-default port
database_name = "movieservice"
table = "actor"
user = "nguyentot"
password = "Tot2002@"

url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"

actor_DF = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

actor_DF.show(5)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-99239780918214>, line 19
      8 password = "Tot2002@"
     10 url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"
     12 actor_DF = (spark.read
     13   .format("jdbc")
     14   .option("driver", driver)
     15   .option("url", url)
     16   .option("dbtable", table)
     17   .option("user", user)
     18   .option("password", password)
---> 19   .load()
     20 )
     22 actor_DF.show(5)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/r

#### create mount to bronze


In [0]:
%fs ls /mnt/datalake/bronze/mysql

In [0]:
# Table name
actor_table = "bronze.actor"

# bronze location
actor_bronzeLocation = "/mnt/datalake/bronze/mysql/actor"

In [0]:
 
from delta.tables import *

#check if the bronze location contain the delta table
# if (DeltaTable.isDeltaTable(spark, actor_bronzeLocation)):
#     # If Delta table exists, append the data

#     # In the bronze zone, we do not do any filtering or remove duplicates
#     actor_DF.write.mode("append").format("delta").save(actor_bronzeLocation)
# else:
#     # If no, save the file to bronzeLocation
actor_DF.write.mode("overwrite").format("delta").save(actor_bronzeLocation)
    

In [0]:
# create the schema and table, if required
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {actor_table} USING delta LOCATION '{actor_bronzeLocation}'")

# Note: Using spark.sql because we use f-string to retrieve the processedLocation

In [0]:
%sql
DESCRIBE HISTORY bronze.actor;

In [0]:
# Maintenance for Delta table

# To optimized the performance of the Delta table, we need to excute 2 commands:
# 1. optimize(): Optimize the number of files used to store the data
# 2. vacuum(): Remove the old version of the data. This reduces the overhead but it limites the version we can go back to.
actor_DataDelta = DeltaTable.forName(spark, actor_table)

if actor_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	actor_DataDelta.optimize()
	actor_DataDelta.vacuum()

In [0]:
%sql
DESCRIBE HISTORY bronze.actor;